# Pictorial Stacked Chart plot
### Chart plotly test

In [1]:
#pip install pandas -q
#pip install plotly -q
#pip install Pillow -q
#pip install cairosvg -q

In [1]:
import plotly.express as px
import PIL
import cairosvg
import plotly.express as px
import io
import numpy as np
import pandas as pd

#Function generate color scale in array svg
def genCategoryColor(matrix,start = 0, end = 1, color_rgb=[255,0,0]):
    for x in range(int(round(matrix.shape[0]*start,0)), int(round(matrix.shape[0]*end,0))):
        if (np.any(matrix[x,:,:])):  
            for y in range(matrix.shape[1]): 
                if (np.any(matrix[x][y])):
                    if x>=matrix.shape[0]*start and x < matrix.shape[0]*end:
                        matrix[x][y] = [color_rgb[0],color_rgb[1],color_rgb[2],list(matrix[x][y])[-1]]
                        # x_l.append(x)
                        # y_l.append(y)
                        # color_l.append()
    return matrix

#Function generate annotation in chart
def addAnnotation(fig,shape, start= 0, end = 1, text='Annotation'):
     fig.add_annotation(
        x=0,
        y=shape[0]*((start+end)/2),
        text=text,
        showarrow=True,
        xanchor="right",
        ax=-50,
        ay=0,
        arrowsize=0.5,
    )


#data example:
pc_1 = {'name':'Python', 'start':0, 'end': 0.34, 'color':[0, 150, 68]}
pc_2 = {'name':'Javascript', 'start':0.34, 'end': 0.57, 'color':[218, 175, 0]}
pc_3 = {'name':'Java', 'start':0.57, 'end': 0.67, 'color':[0, 159, 218 ]}
pc_4 = {'name':'Otros', 'start':0.67, 'end': 1, 'color':[122, 122, 122]}
df = pd.DataFrame([pc_1,pc_2,pc_3,pc_4])

# svg icon from https://fontawesome.com/search?m=free&o=r and generate matrix
svg = '<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 320 512"><!--!Font Awesome Free 6.6.0 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2024 Fonticons, Inc.--><path d="M112 48a48 48 0 1 1 96 0 48 48 0 1 1 -96 0zm40 304l0 128c0 17.7-14.3 32-32 32s-32-14.3-32-32l0-223.1L59.4 304.5c-9.1 15.1-28.8 20-43.9 10.9s-20-28.8-10.9-43.9l58.3-97c17.4-28.9 48.6-46.6 82.3-46.6l29.7 0c33.7 0 64.9 17.7 82.3 46.6l58.3 97c9.1 15.1 4.2 34.8-10.9 43.9s-34.8 4.2-43.9-10.9L232 256.9 232 480c0 17.7-14.3 32-32 32s-32-14.3-32-32l0-128-16 0z"/></svg>'
im = PIL.Image.open(io.BytesIO(cairosvg.svg2png(bytestring=svg))).convert("RGBA")
matrix = np.array(im)

for index, row in df.iterrows():
  matrix = genCategoryColor(matrix,row['start'],row['end'],list(row['color']))

fig = px.imshow(
    matrix,
    y= range(matrix.shape[0]),
    x= range(matrix.shape[1]),
    height=200,
    width=400,
    template='none',
    labels=dict(x="X", 
                y="Y", 
                color="Color")
    )


for index, row in df.iterrows():
    addAnnotation(fig,matrix.shape,row['start'],row['end'],str(round((row['end']-row['start'])*100,2))+'% '+row['name'])

fig.update_xaxes(title = 'Time', showticklabels=False, visible = False)
fig.update_yaxes(visible=False, type='linear', showticklabels=False)
fig.update_layout(coloraxis_showscale=False,margin=dict(l=20,r=20,b=20,t=20))
fig.update_layout(coloraxis_colorbar_x=0.8)
fig.show()